In [4]:
import numpy as np
from pathlib import Path
import zipfile

In [20]:
#resolution = [32, 64, 128, 256]
resolution = [128]
extensions = [['.bing', '.vox', '.rle', '.qstack'], ['.ply', '.xyz', '.binp'], ['.obj', '.stl', '.binm']]
headers = [['Binary raw grid', 'MagicaVoxel', 'RLE', 'QuadStack'], ['PLY', 'XYZ', 'Compressed binary'], ['OBJ', 'STL', 'Binary']]

In [6]:
folder = 'D:/allopezr/Fragments/Vessels_renamed/'

In [23]:
footprint_compressed = np.zeros((len(resolution), len(extensions), 4))
footprint_uncompressed = np.zeros((len(resolution), len(extensions), 4))
count_files = np.zeros((len(resolution), len(extensions), 4))

for res_idx, res in enumerate(resolution):
    for data_type_idx, data_type in enumerate(extensions):
        for extension_idx, extension in enumerate(data_type):
            # Search recursively for files with the given extension
            files = [f for f in Path(folder).rglob('*' + str(res) + 'r*' + extension + '.zip')]
            print(f'Found {len(files)} files with extension {extension}')
            
            for file in files:
                if zipfile.is_zipfile(file):
                    with zipfile.ZipFile(file, 'r') as zip_ref:
                        uncompressed_size = 0
                        for info in zip_ref.infolist():
                            uncompressed_size += info.file_size
                        compressed_size = file.stat().st_size
                        
                        footprint_compressed[res_idx, data_type_idx, extension_idx] += compressed_size
                        footprint_uncompressed[res_idx, data_type_idx, extension_idx] += uncompressed_size
                        count_files[res_idx, data_type_idx, extension_idx] += 1
                        
# From bytes to MB
footprint_compressed /= 1024**2
footprint_uncompressed /= 1024**2

average_footprint_compressed = footprint_compressed / count_files
average_footprint_uncompressed = footprint_uncompressed / count_files

Found 204 files with extension .bing
Found 185 files with extension .vox
Found 261 files with extension .rle
Found 264 files with extension .qstack
Found 1007 files with extension .ply
Found 1185 files with extension .xyz
Found 1344 files with extension .binp
Found 1049 files with extension .obj
Found 1012 files with extension .stl
Found 1016 files with extension .binm
[[[4.00001144 8.00119781 0.36544699 0.13029211]
  [0.09392738 0.10389639 0.0120733         nan]
  [0.1846837  1.04802037 0.22439834        nan]]]


C:\Users\allopezr\AppData\Local\Temp\4\ipykernel_20196\4220358823.py:28: RuntimeWarning: invalid value encountered in divide
  average_footprint_compressed = footprint_compressed / count_files
C:\Users\allopezr\AppData\Local\Temp\4\ipykernel_20196\4220358823.py:29: RuntimeWarning: invalid value encountered in divide
  average_footprint_uncompressed = footprint_uncompressed / count_files


In [18]:
footprint_compressed

array([[[  1.65214062, 228.09575176,   0.9117775 ,   1.41644287],
        [ 22.60584641,  20.25100136,   4.91166878,   0.        ],
        [ 29.78444004,  89.72231293,  25.15251446,   0.        ]]])

In [26]:
latex_table_str = ''

for data_type_idx, data_type in enumerate(headers):
    for extension_idx, extension in enumerate(data_type):
        latex_table_str += '\textbf{' + headers[data_type_idx][extension_idx] + '} & '
        
        for res_idx, res in enumerate(resolution):
            latex_table_str += f'{footprint_compressed[res_idx, data_type_idx, extension_idx]:.2f}MB ({footprint_uncompressed[res_idx, data_type_idx, extension_idx]:.2f}MB) & '
            latex_table_str += f'{average_footprint_compressed[res_idx, data_type_idx, extension_idx]:.2f}MB ({average_footprint_uncompressed[res_idx, data_type_idx, extension_idx]:.2f}MB) & '
        
        latex_table_str = latex_table_str[:-2] + '\\ \n'
    latex_table_str = latex_table_str[:-2] + '\\ \cmidrule{1-9}\n'
latex_table_str = latex_table_str[:-1]

In [27]:
latex_table_str

'Binary raw grid & 4.44MB (816.00MB) & 0.02MB (4.00MB) \\ \nMagicaVoxel & 479.70MB (1480.22MB) & 2.59MB (8.00MB) \\ \nRLE & 4.37MB (95.38MB) & 0.02MB (0.37MB) \\ \nQuadStack & 7.81MB (34.40MB) & 0.03MB (0.13MB) \\\\ \\cmidrule{1-9}\nPLY & 50.74MB (94.58MB) & 0.05MB (0.09MB) \\ \nXYZ & 53.61MB (123.12MB) & 0.05MB (0.10MB) \\ \nCompressed binary & 15.30MB (16.23MB) & 0.01MB (0.01MB) \\\\ \\cmidrule{1-9}\nOBJ & 69.96MB (193.73MB) & 0.07MB (0.18MB) \\ \nSTL & 200.59MB (1060.60MB) & 0.20MB (1.05MB) \\ \nBinary mesh & 57.08MB (227.99MB) & 0.06MB (0.22MB) \\\\ \\cmidrule{1-9}'